# Moneyball

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline

In [2]:
all_master = pd.read_csv('data/Master.csv')
all_salaries = pd.read_csv('data/Salaries.csv')
all_appearances = pd.read_csv('data/Appearances.csv')
all_batting = pd.read_csv('data/Batting.csv')
batting = all_batting[['playerID', 'yearID', 'teamID', 
                       'G', 'H', 'BB', 'HBP', 'AB', 'SF']]
batting = batting[batting['SF'] > 0]
batting = batting[batting['G'] > 20]
batting['OBP'] = ((batting['H'] + batting['BB'] + batting['HBP'])/
                  (batting['AB'] + batting['BB'] + batting['HBP']
                   + batting['SF']))
master = all_master[['playerID', 'nameFirst', 'nameLast']]
appearances = all_appearances[['playerID', 'yearID', 
                               'G_p', 'G_c', 'G_1b', 
                               'G_2b', 'G_3b', 'G_ss', 
                               'G_lf', 'G_cf', 'G_rf']]
salaries = all_salaries[['playerID', 'yearID', 'salary']]
merged = pd.merge(pd.merge(pd.merge(master, batting), salaries), appearances)
merged['worth'] = merged['OBP'] / merged['salary'] * 1000000


In [3]:
positions = ['G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 
             'G_ss', 'G_lf', 'G_cf', 'G_rf']
def build_team(year=2014):
    trimmed = merged[merged['yearID'] == year]
    trimmed = trimmed.sort('worth', ascending=False)
    team = {}
    for i in trimmed.index:
        for pos in positions:
            if trimmed.loc[i, pos] > 0 and pos not in team:
                team[pos] = [trimmed.loc[i, 'nameFirst'],
                             trimmed.loc[i, 'nameLast']]
                break
    return team

In [4]:
moneyball_team = build_team(year=2001)
for player in moneyball_team:
    print(player.ljust(4), " - ", moneyball_team[player][0], moneyball_team[player][1])

G_rf  -  Brian Buchanan
G_p   -  Desi Relaford
G_3b  -  Frank Menechino
G_lf  -  Mike Darr
G_2b  -  David Eckstein
G_1b  -  Albert Pujols
G_cf  -  Juan Pierre
G_ss  -  D'Angelo Jimenez
G_c   -  Shawn Wooten


OBP = (Hits + Bases on Balls(walks) + Hit By Pitch)/(At Bats + Bases on Balls(walks) + Hit By Pitch + Sacrifice Flies)

## Results

With this moneyball algorithm, we take in data on batting, salaries, and positions played.  We calculate the On Base Percentage and then the OBP per dollar.  Giving the program a year, we can get a team that should perform reasonably well while not costing as much.